In [49]:
"""
Dependency
"""
!pip install graphviz
!apt-get -q -y install graphviz libgraphviz-dev pkg-config
!pip install pygraphviz==1.6
import re
import itertools
import random

dep = []
dep.append(["ROOT","S","ROOT","S"])

class Graph:
    """
    Dependency Graph
    ---------------------
    members: child, nodes
    functions: add_edge() , print_child()
    
    """
    def __init__(self):
        """
        Initialize all the member 
        """
        self.child = {}
        self.nodes = []
    def add_edge(self,u,v):
        """
        Add the edge to the dependency Graph
        """

        if u not in self.child:
            self.child[u] = [v]
        else:
            self.child[u].append(v)
            
    def print_child(self):
        """
        print the adjacency list
        """
        for n in self.child:
            print(n,"=>",self.child[n])


class Node:
    
    def __init__(self,data):
        """

        Initialize all the member
        """
        self.data = data
        self.Head = None
        self.parent = None
        self.child = []
        """
            this is the head percolation dictionary
        """
        self.phrase = {}
        self.phrase['ROOT'] = ["l","S|SQ|SINV|SBARQ"]
        self.phrase['ADJP'] = ["r","JJ*|VB*|NN*;ADJP;IN;RB|ADVP;CD|QP;FW|NP;*"]
        self.phrase['ADVP'] = ["r"," VB*;RP;RB*|JJ*;ADJP;ADVP;QP;IN;NN;CD;NP;*"]
        self.phrase['CONJP'] = ["l","CC;VB*;NN*;TO|IN;*"]
        self.phrase['EDITED'] = ["r","VP;VB*;NN*|PRP|NP;IN|PP;S*;*"]
        self.phrase['EMBED'] = ["r","S*;FRAG|NP;*"]
        self.phrase['FRAG'] = ["r","VP;VB*;-PRD;S|SQ|SINV|SBARQ;NN*|NP;PP;SBAR;JJ*|ADJP;RB|ADVP;INTJ;*"]
        self.phrase['INTJ'] = ["l","VB*;NN*;UH;INTJ;*"]
        self.phrase['LST'] = ["l"," LS|CD;NN;*"]
        self.phrase['META'] = ["l","NP;VP|S;*"]
        self.phrase['NAC'] = ["r","NN*;NP;S|SINV;*"]
        self.phrase['NML'] = ["r","NN*|NML;CD|NP|QP|JJ*|VB*;*"]
        self.phrase['NX'] = ["r","NN*;NX;NP;*"]
        self.phrase['NP'] = ["r","NN*|NML;NX;PRP;FW;CD;NP;-NOM;QP|JJ*|VB*;ADJP;S;SBAR;*"]
        self.phrase['PP'] = ["l","RP;NP;TO;IN;VB*;PP;NN*;JJ;RB;*"]
        self.phrase['QP'] = ["r","CD;NN*;JJ;DT|PDT;RB;NP|QP;*"]
        self.phrase['RRC'] = ["l","VP;VB*;-PRD;NP|NN*;ADJP;PP;*"]
        self.phrase['S'] = ["r","VP;VB*;-PRD;S|SQ|SINV|SBARQ;SBAR;NP;PP;*"]
        self.phrase['SBAR'] = ["r","VP;S|SQ|SINV;SBAR*;FRAG|NP;*"]
        self.phrase['SBARQ'] = ["r","VP;SQ|SBARQ;S|SINV;FRAG|NP;*"]
        self.phrase['SQ'] = ["r","VP;VB*;SQ;S;MD;NP;*"]
        self.phrase['SINV'] = ["r","VP;VB*;MD;S|SINV;NP;*"]
        self.phrase["UCP"] = ["l","*"]
        self.phrase['VP'] = ["l","VP;VB*;MD|TO;JJ*|NN*|IN;-PRD;NP;ADJP|QP;S;*"]
        self.phrase['WHADJP'] = ["r","JJ*|VBN;WHADJP|ADJP;*"]
        self.phrase['WHADVP'] = ["r","RB*|WRB;WHADVP;*"]
        self.phrase['WHNP'] = ["r","NN*;WP|WHNP;NP|NML|CD;JJ*|VBG;WHADJP|ADJP;DT;*"]
        self.phrase['WHPP'] = ["l","IN|TO;*"]
        self.phrase['X'] = ["r","*"]
        
        
    def add(self,val):
        """
        Add new node to the tree

        """
        Newnode = Node(val)
        #print("Added {} to {}".format(Newnode.data,self.data))
        Newnode.parent = self
        self.child.append(Newnode)
        return Newnode
    
    def subtree(self,x):
        
        parent = self
        for i in x:
            #print("value",i,self.data)
            if(i == "("):
                node = self.add(next(x))
                #for b in node.child:
                    #print("child",b)
                ret = node.subtree(x)
                
            elif i == ")":
                #print("this")
                #self.printtree()
                #print("parent",parent.data)
                return parent
                
            else:
                #print("running",self.data,i)
                self.add(i)
                
        
                    
    def printtree(self,G):
        if self.child:
            print(self.data,"=>",end=" ")
            for i in self.child:
                print(i.data,end = " ")
            print()    
                #i.printtree()
            for i in self.child:
                #print(i.data,end = " ")
                i.printtree(G)
        else:
            G.nodes.append(self.data)
            #print("no child",self.data)
    def fHead(self,A,Y,Z,phrase):
        
        S = A.data
        #print("finding Head",S)
        X = Y.copy()
        if S in Z:
            #print("Pos tag")
            A.Head = A.child[0].data 
            return A.child[0].data
        if(phrase[S][0]=='r'):
            X.reverse()
        

        terms = phrase[S][1].split(";")
        #print(j,i[0],terms)
        for tag in terms:
            #print(tag)
            tag = re.split("\|",tag)
            #print(tag,"with |")
            for w in X:
                #print(w.data,"word")
                for t in tag:
                    #print(t,w.data,"hello")
                    if t is "*" or re.search("^"+t,w.data):
                        #print(t,w.data,"in X")
                        #print(w.Head)
                        return w.Head
                    


        
    
    def findDependent(self,Z):
        #print("Hello")
        if self.child:
            
            for i in self.child:
                i.findDependent(Z)
            
            
            self.Head = self.fHead(self,self.child,Z,self.phrase)
            
            #print(self.data,"=>",self.Head)
    
    def createDependency(self,G):
        if self.child:
            for i in self.child:
                if self.Head != i.Head and i.Head:
                    print(self.Head,"=>",i.Head,end="\t>> ")
                    G.add_edge(self.Head,i.Head)
                    labelling1(self,i)
                i.createDependency(G)
def labelling1(self,i):
        P = self.data
        C = i.data
        Pi = self.Head
        Ci = i.Head
        if P == "ROOT":
            if(C in ["S","SINV"]):
                print(Pi,"{ROOT_S}",Ci)
            elif(C in ['SQ']):
                print(Pi,"{ROOT_SQ}",Ci)
            elif C in ['SBARQ']:
                print(Pi,"{ROOT_SBARQ}",Ci)

        #3
        elif hasAdverbialTag(C):
            #print("has adverbial")
            if C in "S|SBAR|SINV".split("|"):
                print(Pi,"{ADVCL}",Ci)
            if C in "NML|NP|QP".split("|"):
                print(Pi,"{NPADVMOD}",Ci)
        
        
            
        #10
        elif P in "VP|SINV|SQ".split("|"):
            if C is "ADJP":
                print(Pi,"{ACOMP}",Ci)
            elif C in "MD|TO".split("|") or (C in "VB|VBP|VBZ|VBN|VBG|VBD".split("|")):
                print(Pi,"{AUX}",Ci)
            elif (getObjectLabel(C)):
                print(Pi,"{",getObjectLabel(C),"}",Ci)
            else:
                print(Pi,"{VMOD}",Ci)
        elif C in ['PRN']:
            print(Pi,"{PRN}",Ci)
        elif C in ['DT']:
            print(Pi,"{DET}",Ci)
        elif C in ['PP','ADVP','SBAR'] and P in ['VP']:
            print(Pi,"{ADV}",Ci)
        elif P in ['VP','S','SBAR','SBARQ','SINV','SQ']:
            print(Pi,"{VMOD}",Ci)
        elif C in ['VP'] and P in ['VP','SQ','SINV']:
            print(Pi,"{VC}",Ci)
        
        
        #29
        elif C in "S|SBARQ|SINV|SQ".split("|"):
            print(Pi,"{CCOMP}",Ci)
        #30
        elif P in ['QP']:
            if C in ['CD']:
                print(Pi,"{NUMBER}",Ci)
                return
            print(Pi,"{QUANTMOD}",Ci)
        #31
        elif (P in "NML|NP|NX|WHNP|NAC".split("|")) or P in "NN|NNP|NNS|PRP|WP".split("|"):
            print(Pi,"{",getNounModifierLabel(C,P),"}",Ci)
        #33
        elif (getSimpleLabel(C,Ci,P)):
            print(Pi,"{"+getSimpleLabel(C,Ci,P)+"}",Ci)
        elif P in ['ADJP','ADVP','WHADJP','WHADVP','PP']:
            if C in "NML|NP|QP|NN|NNS|NNP|PRP|WP".split("|"):
                print(Pi,"{NPADVMOD}",Ci)
                return
            print(Pi,"{AMOD}",Ci)
        
        elif P in ['PP','WHPP']:
            print(Pi,"{PMOD}",Ci)
        else:
            print(Pi,"{DEP}",Ci)

def getSimpleLabel(C,Ci,P):
    if C is "HYPH":
        return "HYPH"
    if C in "ADJP|WHADJP|JJ|JJR".split("|"):
        return "AMOD"
    if C in "PP|WHPP".split("|"):
        return "PREP"
    if C in "PRT|RP".split("|"):
        return "PRT"
    if isPreCorrelativeConjunction(C,Ci):
        return "PRECONJ"
    if isCoordinatingConjunction(C):
        return "CC"
    if isParentheticalModifier(C):
        return "PARATAXIS"
    if isPunctuation(C):
        return "PUNCT"
    if isInterjection(C):
        return "INTJ"
    if isAdverbialModifier(C,P):
        return "ADVMOD"
    return None
def hasAdverbialTag(C):
    if C in ['ADV','BNF','DIR','EXT','LOC','MNR','PRP','TMP','VOC']:
        return True
    return False
def isAdverbialModifier(C,P):
    if C in "ADVP|RB|RBS|RBR|WRB".split("|"):
        if P in ['PP']:
            return False
    return True
def isPunctuation(C):
    if C in ":|,|.|“|”|-LRB-|-RRB-|HYPH|NFP|SYM|PUNC".split("|"):
        return True
def isPossessionModifier (C,P):
    if C in ['PRP$']:
        return True
    if P in "ADJP|NML|NP|QP|WHNP ".split("|"):
        return True
    return False
def isParentheticalModifier(C):
    return C in ['PRN']
def isInterjection(C):
    return C in "INTJ|UH".split("|")
def getNounModifierLabel(C,P):
    if C in "VBG|VBN".split("|"):
        return "AMOD"
    if C in "DT|WDT|WP".split("|"):
        return "DET"
    if C in ["PDT"]:
        return "PREDET"
    if C in "NML|NP|FW|NN*".split("|"):
        return "NN"
    if C in "CD|QP".split("|"):
        return "NUM"
    if C in ["POS"]:
        return "POSSESSIVE"
    return "NMOD"
def isPreCorrelativeConjunction(C,Ci):
    if C in ["CC"] and Ci in "both|either|neither|whether".split("|"):
        return True
    if C in ['CONJP'] and Ci in ["not only"]:
        return True
    return False
def getObjectLabel(C):
    if C in "NP|NML".split("|"):
        return "DOBJ"
    return None
def isCoordinatingConjunction(C):
    return C in "CC|CONJP".split("|") 

In [54]:
def getCP():
    C = []
    C.append("(S (NP (NNS Students)) (VP (VBP play) (NP (NN football))))")
    C.append("(S (NP (NNS Students)) (VP (VBP are) (VP (VBG playing) (ADVP (RB very) (RB well)))))")
    C.append("(S (NP (PRP He)) (VP (VBZ is) (ADJP (RB very) (JJ intelligent))))")
    C.append("(S (NP (NP (DT the) (NN person)) (SBAR (WHNP (WP whom)) (S (NP (PRP i)) (VP (VBD met))))) (VP (VBZ is) (NP (NNP shubham))))")
    C.append("(S (SBAR (IN If) (S (NP (PRP I)) (VP (VBP bring) (NP (CD 10) (NNS dollars)) (NP (NN tomorrow))))) (, ,) (MD can) (NP (PRP you)) (VP (VB buy) (NP (PRP me)) (NP (NN lunch))) (. ?))")
    C.append("(S (NP (PRP She)) (VP (VBZ considers) (S (NP (PRP me)) (NP (PRP$ her) (NN friend)))))")
    C.append("(NP (DT The) (NML (NNP Zhuhai) (HYPH -) (NNP Hong) (NNP Kong) (HYPH -) (NNP Macao)) (NN bridge))")
    C.append("(ROOT (S (NP (PRP$ My) (NN dog)) (ADVP (RB also)) (VP (VBZ likes) (S (VP (VBG eating) (NP (NN sausage))))))")
    C.append("(ROOT (S (NP (PRP i)) (VP (VBP am) (VP (VBG talking) (PP (IN to) (NP (PRP you))) (ADVP (RB only))))))")
    C.append("(ROOT (SBARQ (WHNP (WP what)) (SQ (MD should) (NP (PRP I)) (VP (VB do)))))")
    C.append("(ROOT (S (VP (VB Answer) (NP (DT the) (VBG following) (NN question)))))")
    C.append("(ROOT (S (NP (DT this)) (VP (VBZ is) (RB not) (SBAR (WHNP (WP what)) (S (NP (PRP you)) (VP (VBP think) (FRAG (VBZ is))))))))")
    C.append("(S (NP (PRP I)) (VP (VBP prefer) (NP (NP (DT the) (NN morning) (NN flight)) (PP (IN through) (NP (NNP Denver))))))")
    C.append("(S (NP (NNP Vinken)) (VP (MD will) (VP (VB join) (NP (DT the) (NN board)) (PP (IN as) (NP (DT a) (JJ nonexecutive) (NN director))) (NP (NNP Nov) (CD 29)))))")
    i = random.randint(0,11)
    return C[-2]
def CP_to_DP():
    CP = getCP()
    print("***Constituency Parse Tree***\n\n",CP)
    CP = CP.replace("(","( ")
    CP = CP.replace(")"," )")
    X = CP.split(" ")

    x = iter(X)
    if(not next(x)):
        return False

    S = Node(next(x))
    S.subtree(x)
    return S


In [55]:
POS_TAG = ['CC','CD','DT','EX','FW','NN', 'WP', 'PRP', 'VBD', 'VBZ',
           'NNP','HYPH','.','NNS','VBP','VBG','RB','VB','PRP$','PRP',
           'MD','JJ','VBZ','IN','CD',',','JJR','JJS','LS','PDT','NNPS',
            'POS','RBR','RBS','RP','SYM','TO','UH','VB','VBN','WDT','WP','WP$','WRB']

S  = CP_to_DP()
print("\n***Tree Construction***\n")
G = Graph()
S.printtree(G)
print("\n***String is :\n",G.nodes)

***Constituency Parse Tree***

 (S (NP (PRP I)) (VP (VBP prefer) (NP (NP (DT the) (NN morning) (NN flight)) (PP (IN through) (NP (NNP Denver))))))

***Tree Construction***

S => NP VP 
NP => PRP 
PRP => I 
VP => VBP NP 
VBP => prefer 
NP => NP PP 
NP => DT NN NN 
DT => the 
NN => morning 
NN => flight 
PP => IN NP 
IN => through 
NP => NNP 
NNP => Denver 

***String is :
 ['I', 'prefer', 'the', 'morning', 'flight', 'through', 'Denver']


In [56]:

S.findDependent(POS_TAG)
print("\n***Dependency Tree***\n")

S.createDependency(G)



***Dependency Tree***

prefer => I	>> prefer {VMOD} I
prefer => flight	>> prefer { DOBJ } flight
flight => the	>> flight {DET} the
flight => morning	>> flight { NMOD } morning
flight => Denver	>> flight { NMOD } Denver
Denver => through	>> Denver {ADVMOD} through


In [57]:
import pygraphviz as pgv

X = pgv.AGraph()
for node in G.child:
    for i in G.child[node]:
        X.add_edge(node,i,)
print(X.string())  # print to screen
print("Wrote simple.dot")
X.write("simple.dot")  # write to simple.dot

B = pgv.AGraph("simple.dot")  # create a new graph from file
B.layout()  # layout with default (neato)
B.draw("simple.png",format = 'png',prog='dot')  # draw png
print("Wrote simple.png")


strict graph "" {
	prefer -- I;
	prefer -- flight;
	flight -- the;
	flight -- morning;
	flight -- Denver;
	Denver -- through;
}

Wrote simple.dot
Wrote simple.png


Reading package lists...
Building dependency tree...
Reading state information...
pkg-config is already the newest version (0.29.1-0ubuntu2).
graphviz is already the newest version (2.40.1-2).
libgraphviz-dev is already the newest version (2.40.1-2).
0 upgraded, 0 newly installed, 0 to remove and 14 not upgraded.


# New Section